In [123]:
import matplotlib.pyplot as plt
#from PIL import Image
import matplotlib.image as mpimg
import numpy as np
#from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras import layers, models
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from tensorflow.keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import tensorflow.metrics
import pandas as pd
from sklearn.model_selection import train_test_split
import os, shutil, random
from sklearn.metrics import confusion_matrix
import itertools

/Users/leona/anaconda3/envs/finalpro/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/leona/anaconda3/envs/finalpro/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/leona/anaconda3/envs/finalpro/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/leona/anaconda3/envs/finalpro/lib

In [3]:
meta = pd.read_csv('data/HAM10000_metadata.csv')

### Identify lesion_IDs with multiple images for the validation set
To ensure that we don't have the same images in the train and validation set

In [4]:
uni = meta.groupby('lesion_id').count()
uni = uni[uni['image_id'] == 1]
unique_list = list(uni.index)

In [5]:
# define function to identify if a lesion id has duplicate images
def is_duplicate(x):
    
    unique_list = list(uni.index)
    
    if x in unique_list:
        return 'no_duplicates'
    else:
        return 'has_duplicates'

In [6]:
meta['duplicate'] = meta['lesion_id']

In [7]:
meta['duplicate'] = meta['duplicate'].apply(is_duplicate)

In [8]:
meta_uni = meta[meta.duplicate == "no_duplicates"]

In [34]:
meta_uni.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,duplicate
10,HAM_0001396,ISIC_0025276,bkl,histo,55.0,female,trunk,no_duplicates
15,HAM_0007207,ISIC_0031326,bkl,histo,65.0,male,back,no_duplicates
20,HAM_0006071,ISIC_0032343,bkl,histo,70.0,female,face,no_duplicates
33,HAM_0005612,ISIC_0024981,bkl,histo,80.0,male,scalp,no_duplicates
34,HAM_0005388,ISIC_0027815,bkl,histo,80.0,male,chest,no_duplicates


In [22]:
diagnose = list(meta['dx'].unique())

In [17]:
train_dir = "/Users/leona/PythonWork/Github_Projects/Final_Pro/data/ISIC2018_Task3_Training_Input/train/"

In [24]:
aug_list = []
for d in diagnose:
    aug_list.append(os.listdir(train_dir + d))

In [29]:
aug_list = [item for sublist in aug_list for item in sublist]

In [90]:
len(aug_list)

38890

In [36]:
meta = meta[['image_id', 'dx', 'age', 'sex', 'lesion_id']]

In [37]:
img_id = list(meta.image_id)

In [47]:
meta['img_nr'] = [image[5:] for image in meta.image_id]

In [48]:
meta.head()

,image_id,dx,age,sex,lesion_id,img_nr
0,ISIC_0027419,bkl,80.0,male,HAM_0000118,0027419
1,ISIC_0025030,bkl,80.0,male,HAM_0000118,0025030
2,ISIC_0026769,bkl,80.0,male,HAM_0002730,0026769
3,ISIC_0025661,bkl,80.0,male,HAM_0002730,0025661
4,ISIC_0031633,bkl,75.0,male,HAM_0001466,0031633


In [79]:
def extract_id(x):
        
    if "ISIC" in x:
        return x[5:12]
    else:
        return  x[:7]

In [83]:
aug_img = pd.DataFrame(aug_list, columns=['aug_id'])
#aug_img["id_nr"] = aug_img.aug_id

In [84]:
aug_img['img_nr'] = aug_img['aug_id'].apply(extract_id)

In [92]:
complete_train = pd.merge(aug_img, meta, how='inner', on="img_nr", left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=('_x', '_y'), copy=True, indicator=False,
         validate=None)

In [96]:
complete_train.head()

,aug_id,img_nr,image_id,dx,age,sex,lesion_id
0,ISIC_0024306.jpg,0024306,ISIC_0024306,nv,45.0,male,HAM_0000550
1,ISIC_0024307.jpg,0024307,ISIC_0024307,nv,50.0,male,HAM_0003577
2,ISIC_0024308.jpg,0024308,ISIC_0024308,nv,55.0,female,HAM_0001477
3,ISIC_0024309.jpg,0024309,ISIC_0024309,nv,40.0,male,HAM_0000484
4,0024310_0_5393445.jpeg,0024310,ISIC_0024310,mel,60.0,male,HAM_0003350


### Support Vector Machine

In [93]:
from sklearn.svm import SVC

In [113]:
copy = complete_train[['age', 'sex', 'dx']]

In [115]:
copy = copy.dropna()
copy.shape

(38801, 3)

In [94]:
m = SVC(kernel='rbf', gamma=0.1, C=0.1)

In [116]:
X = copy[['age', 'sex']]

In [117]:
X.sex = pd.get_dummies(X.sex)

/Users/leona/anaconda3/envs/finalpro/lib/python3.6/site-packages/pandas/core/generic.py:5209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [118]:
y = copy.dx

### Other model example

In [ ]:
filepath = "model_23.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_top_3_accuracy', verbose=1, 
                             save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.5, patience=2, 
                                   verbose=1, mode='max', min_lr=0.00001)
# Create new log_data
tensorboard_callback = tensorflow.keras.callbacks.TensorBoard()                    
                              
callbacks_list = [checkpoint, reduce_lr, tensorboard_callback]

history = model.fit_generator(train_batches, steps_per_epoch=train_steps, 
                              class_weight=class_weights,
                    validation_data=valid_batches,
                    validation_steps=val_steps,
                    epochs=30, verbose=1,
                   callbacks=callbacks_list)

### Define custom loss function for weighted categorical crossentropy

In [ ]:
# Source: https://gist.github.com/wassname/ce364fddfc8a025bfab4348cf5de852d



### Merge layers with demographic data

In [121]:
filepath = "model.h5"

In [129]:
x = Dropout(0.25)(x)
predictions = Dense(7, activation='softmax')(x)

In [130]:
model_dem = Model(inputs=mobile.input, outputs=predictions)

In [133]:
model_dem.load_weights(filepath)

In [136]:
model_dem.layers[-6:]

In [137]:
model_dem.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128   

In [140]:
x = model_dem.layers[-3].outputs()

AttributeError: 'GlobalAveragePooling2D' object has no attribute 'outputs'